In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
import sys
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import train_test_split
from embedding_layers import SkeletalInputEmbedding
from encoder_layers import TransformerEncoder
from decoder_layers import TransformerDecoder
import TF_helper_functions as hf
import pickle


In [29]:
datapath='/home/maleen/research_data/Transformers/datasets/training/'
# Base path and file information
base_name='24_07_25_training_norm'

filename=datapath+base_name+'.pkl'

def load_results_from_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def process_all_datasets(results, input_length=60, predict_length=60):
    all_X_pos, all_X_vel, all_X_acc = [], [], []
    all_Y_pos, all_Y_vel, all_Y_acc = [], [], []
    discarded_frames = {}

    for i in range(1, 7):  # Assuming you have 6 datasets
        dataset_key = f'dataset{i}'
        norm_pos = results[f'{dataset_key}_normpos']
        norm_vel = results[f'{dataset_key}_normvel']
        norm_acc = results[f'{dataset_key}_normacc']

        # Generate sequences for this dataset
        X_pos, X_vel, X_acc, Y_pos, Y_vel, Y_acc = hf.generate_sequences(norm_pos, norm_vel, norm_acc, input_length, predict_length)
        
        all_X_pos.append(X_pos)
        all_X_vel.append(X_vel)
        all_X_acc.append(X_acc)
        all_Y_pos.append(Y_pos)
        all_Y_vel.append(Y_vel)
        all_Y_acc.append(Y_acc)

        # Calculate discarded frames
        total_frames = norm_pos.shape[0]
        used_frames = X_pos.shape[0] + input_length + predict_length - 1
        discarded = total_frames - used_frames
        discarded_frames[dataset_key] = discarded

    # Combine sequences from all datasets
    combined_X_pos = np.concatenate(all_X_pos)
    combined_X_vel = np.concatenate(all_X_vel)
    combined_X_acc = np.concatenate(all_X_acc)
    combined_Y_pos = np.concatenate(all_Y_pos)
    combined_Y_vel = np.concatenate(all_Y_vel)
    combined_Y_acc = np.concatenate(all_Y_acc)

    return (combined_X_pos, combined_X_vel, combined_X_acc, 
            combined_Y_pos, combined_Y_vel, combined_Y_acc, 
            discarded_frames)


In [31]:
input_length = 30
predict_length = 2
datasetnum=6

# Load the results
results = load_results_from_pickle(filename)

# Process all datasets and get combined sequences
(combined_X_pos, combined_X_vel, combined_X_acc, 
 combined_Y_pos, combined_Y_vel, combined_Y_acc, 
 discarded_frames) = process_all_datasets(results, input_length, predict_length)

print("Combined sequences shapes:")
print(f"X_pos shape: {combined_X_pos.shape}")
print(f"X_vel shape: {combined_X_vel.shape}")
print(f"X_acc shape: {combined_X_acc.shape}")
print(f"Y_pos shape: {combined_Y_pos.shape}")
print(f"Y_vel shape: {combined_Y_vel.shape}")
print(f"Y_acc shape: {combined_Y_acc.shape}")

print("\nDiscarded frames per dataset:")
for dataset, frames in discarded_frames.items():
    print(f"{dataset}: {frames} frames")


Combined sequences shapes:
X_pos shape: (7667, 30, 6, 3)
X_vel shape: (7667, 30, 6, 3)
X_acc shape: (7667, 30, 6, 3)
Y_pos shape: (7667, 2, 6, 3)
Y_vel shape: (7667, 2, 6, 3)
Y_acc shape: (7667, 2, 6, 3)

Discarded frames per dataset:
dataset1: 0 frames
dataset2: 0 frames
dataset3: 0 frames
dataset4: 0 frames
dataset5: 0 frames
dataset6: 0 frames
